In [16]:
import numpy as np
import pickle
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Sequential, model_from_json
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
def normalize_data(x):
    print(x.shape)
    sq = np.multiply(x, x)
    mag = np.sqrt(np.sum(sq, axis = 2, keepdims=True))
    #print(mag.shape)    
    #mag = np.concatenate((mag, mag, mag), axis = 2)
    #print(mag.shape)
    x = np.divide(x, mag)
    return x
    

In [18]:
def get_train_data():
    with open("C:\\ASM\\DevData\\eating\\python\\segments.pkl", "rb") as file:
        data = pickle.load(file)
    
    X_train, Y_train = [], []
    for i in range(4):        
        for subject in range(len(data[i])):
            for sess in range(len(data[i][subject])):
                x = data[i][subject][sess]["x"]
                y = data[i][subject][sess]["y"]
                
                if i<2:
                    if len(X_train)==0:
                        X_train = x
                        Y_train = y
                    else:
                        X_train = np.concatenate((X_train, x), axis = 0)
                        Y_train = np.concatenate((Y_train, y), axis = 0)
                        
                elif i==3 and subject <5:
                    x = x[y==0, :, :]
                    y = y[y==0]
                    
                    X_train = np.concatenate((X_train, x), axis = 0)
                    Y_train = np.concatenate((Y_train, y), axis = 0)                    
                
    X_train = normalize_data(X_train)         
    return X_train, Y_train

In [19]:
X_train, Y_train = get_train_data()

(21412, 128, 3)
(21412, 128, 1)


In [20]:
x = X_train[5, :, :]
x = np.sqrt(np.sum(np.multiply(x, x), axis = 1))
print(x)


[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]


In [21]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
Y_train = Y_train.reshape(Y_train.shape[0], 1)
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)

X_train shape:  (21412, 128, 3, 1)
Y_train shape:  (21412, 1)


In [22]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)
batch_size = 64
num_classes = 2
epochs = 50

In [23]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), padding ='valid', strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape ))
model.add(MaxPooling2D(pool_size=(4, 1), strides=(4, 1)))

model.add(Conv2D(64, kernel_size=(2, 2), padding ='valid', strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 1), strides=(4, 1)))


model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=binary_crossentropy,
              optimizer= Adam(),
              metrics=['accuracy'])


model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs)

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Epoch 1/50
21412/21412 [==============================] - 3s 149us/step - loss: 0.3549 - acc: 0.8557
Epoch 2/50
21412/21412 [==============================] - 3s 123us/step - loss: 0.2903 - acc: 0.8834
Epoch 3/50
21412/21412 [==============================] - 3s 125us/step - loss: 0.2720 - acc: 0.8894
Epoch 4/50
21412/21412 [==============================] - 3s 124us/step - loss: 0.2613 - acc: 0.8937
Epoch 5/50
21412/21412 [==============================] - 3s 127us/step - loss: 0.2459 - acc: 0.8998
Epoch 6/50
21412/21412 [==============================] - 3s 126us/step - loss: 0.2359 - acc: 0.9049
Epoch 7/50
21412/21412 [==============================] - 3s 128us/step - loss: 0.2240 - acc: 0.9101
Epoch 8/50
21412/21412 [==============================] - 3s 128us/step - loss: 0.2167 - acc: 0.9136
Epoch 9/50
21412/21412 [==============================] - 3s 132us/step - loss: 0.2060 - acc: 0.9199
Epoch 10/50
21412/21412 [==============================] - 3s 130us/step - loss: 0.1977 - a

In [39]:
with open("C:\\ASM\\DevData\\eating\\python\\segments.pkl", "rb") as file:
    data = pickle.load(file)
    
    X, Y = [], []
    i = 3
    for subject in range(len(data[i])): #
        for sess in range(len(data[i][subject])): #
            X = data[i][subject][sess]["x"]
            Y = data[i][subject][sess]["y"]
            X = normalize_data(X)
            X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
            Y = Y.reshape(Y.shape[0], 1)
            
            scores = model.evaluate(X, Y, verbose=0)
            print("UVA lab Subejct {}, Session {}, score: {}".format(subject, sess, scores[1]*100))
            Ypr = model.predict(X, verbose=0)
            Yprb = Ypr[:, 0]>0.5
            Yprb = Yprb.reshape((len(Y), 1))
            Y = np.concatenate((Y, Yprb, Ypr), axis=1)
            #print(Y)
            
            filepath = "C:\\ASM\\DevData\\eating\\python\\results\\steven_free\\"+str(subject)+"_"+str(sess)+".csv"
            np.savetxt(filepath, Y, delimiter=",")
            #with open(filepath, "r") as file:
            #    file.write(Y)


(1344, 128, 3)
(1344, 128, 1)
UVA lab Subejct 0, Session 0, score: 87.42559523809523
[[  0.00000000e+00   0.00000000e+00   2.63950926e-29]
 [  0.00000000e+00   0.00000000e+00   4.74825981e-07]
 [  0.00000000e+00   0.00000000e+00   3.47047668e-10]
 ..., 
 [  0.00000000e+00   0.00000000e+00   1.78499287e-03]
 [  0.00000000e+00   0.00000000e+00   4.03437497e-08]
 [  0.00000000e+00   0.00000000e+00   1.38480918e-05]]
(1502, 128, 3)
(1502, 128, 1)
UVA lab Subejct 0, Session 1, score: 90.27962716378163
[[  0.00000000e+00   0.00000000e+00   1.55631830e-09]
 [  0.00000000e+00   0.00000000e+00   4.04703755e-08]
 [  0.00000000e+00   0.00000000e+00   1.12936902e-21]
 ..., 
 [  0.00000000e+00   0.00000000e+00   5.90020418e-03]
 [  0.00000000e+00   0.00000000e+00   4.47370065e-03]
 [  0.00000000e+00   0.00000000e+00   2.69245382e-09]]
(2007, 128, 3)
(2007, 128, 1)
UVA lab Subejct 1, Session 0, score: 82.46138515196812
[[  0.00000000e+00   0.00000000e+00   2.62625682e-14]
 [  0.00000000e+00   0.0000